# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


T. Mikal-Evans  ->  T. Mikal-Evans  |  ['T. Mikal-Evans']
Arxiv has 85 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2308.09742


extracting tarball to tmp_2308.09742...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


Found 111 bibliographic references in tmp_2308.09742/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2308.10763


/tmp/ipykernel_2207/3009462391.py:49: LatexWarning: 2308.10763 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.09742-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.09742) | **Ionised gas kinematics and dynamical masses of $z\gtrsim6$ galaxies from  JADES/NIRSpec high-resolution spectroscopy**  |
|| <mark>A. d. Graaff</mark>, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2023-08-22*|
|*Comments*| *Software for JWST/NIRSpec MSA modelling (slit losses, 1D LSFs and 2D model fitting) publicly available at this https URL*|
|**Abstract**| We explore the kinematic gas properties of six $5.5<z<7.4$ galaxies in the JWST Advanced Deep Extragalactic Survey (JADES), using high-resolution JWST/NIRSpec multi-object spectroscopy of the rest-frame optical emission lines [OIII] and H$\alpha$. The objects are small and of low stellar mass ($\sim 1\,$kpc; $M_*\sim10^{7-9}\,{\rm M_\odot}$), less massive than any galaxy studied kinematically at $z>1$ thus far. The cold gas masses implied by the observed star formation rates are $\sim 10\times$ larger than the stellar masses. We find that their ionised gas is spatially resolved by JWST, with evidence for broadened lines and spatial velocity gradients. Using a simple thin-disc model, we fit these data with a novel forward modelling software that accounts for the complex geometry, point spread function, and pixellation of the NIRSpec instrument. We find the sample to include both rotation- and dispersion-dominated structures, as we detect velocity gradients of $v(r_{\rm e})\approx100-150\,{\rm km\,s^{-1}}$, and find velocity dispersions of $\sigma_0\approx 30-70\,{\rm km\,s^{-1}}$ that are comparable to those at cosmic noon. The dynamical masses implied by these models ($M_{\rm dyn}\sim10^{9-10}\,{\rm M_\odot}$) are larger than the stellar masses by up to a factor 40, and larger than the total baryonic mass (gas + stars) by a factor of $\sim 3$. Qualitatively, this result is robust even if the observed velocity gradients reflect ongoing mergers rather than rotating discs. Unless the observed emission line kinematics is dominated by outflows, this implies that the centres of these galaxies are dark-matter dominated or that star formation is $3\times$ less efficient, leading to higher inferred gas masses. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.10763-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.10763) | **Transit Timing Variations in the three-planet system: TOI-270**  |
|| L. Kaye, et al. -- incl., <mark>T. Mikal-Evans</mark> |
|*Appeared on*| *2023-08-22*|
|*Comments*| *22 pages, 7 figures*|
|**Abstract**| We present ground and space-based photometric observations of TOI-270 (L231-32), a system of three transiting planets consisting of one super-Earth and two sub-Neptunes discovered by TESS around a bright (K-mag=8.25) M3V dwarf. The planets orbit near low-order mean-motion resonances (5:3 and 2:1), and are thus expected to exhibit large transit timing variations (TTVs). Following an extensive observing campaign using 8 different observatories between 2018 and 2020, we now report a clear detection of TTVs for planets c and d, with amplitudes of $\sim$10 minutes and a super-period of $\sim$3 years, as well as significantly refined estimates of the radii and mean orbital periods of all three planets. Dynamical modeling of the TTVs alone puts strong constraints on the mass ratio of planets c and d and on their eccentricities. When incorporating recently published constraints from radial velocity observations, we obtain masses of $M_{\mathrm{b}}=1.48\pm0.18\,M_\oplus$, $M_{c}=6.20\pm0.31\,M_\oplus$ and $M_{\mathrm{d}}=4.20\pm0.16\,M_\oplus$ for planets b, c and d, respectively. We also detect small, but significant eccentricities for all three planets : $e_\mathrm{b} =0.0167\pm0.0084$, $e_{c} =0.0044\pm0.0006$ and $e_{d} = 0.0066\pm0.0020$. Our findings imply an Earth-like rocky composition for the inner planet, and Earth-like cores with an additional He/H$_2$O atmosphere for the outer two. TOI-270 is now one of the best-constrained systems of small transiting planets, and it remains an excellent target for atmospheric characterization. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2308.09742.md
    + _build/html/tmp_2308.09742/./figures/sample_cutouts.png
    + _build/html/tmp_2308.09742/./figures/z_vs_sigma_withsubmm.png
    + _build/html/tmp_2308.09742/./figures/exposure_models.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{\rm km s^{-1}}$
$\newcommand{\Msun}{\rm M_\odot}$
$\newcommand{\re}{r_{\rm e}}$
$\newcommand{\rt}{r_{\rm t}}$
$\newcommand{\va}{v_{\rm a}}$
$\newcommand{\vc}{v_{\rm circ}}$
$\newcommand{\vdisp}{\sigma_0}$
$\newcommand{\Mdyn}{M_{\rm dyn}}$
$\newcommand{\Mgas}{M_{\rm gas}}$
$\newcommand{\Mbar}{M_{\rm bar}}$
$\newcommand{\micron}{\rm \mu m}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# Ionised gas kinematics and dynamical masses of $z\gtrsim6$ galaxies from JADES/NIRSpec high-resolution spectroscopy   

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.09742-b31b1b.svg)](https://arxiv.org/abs/2308.09742)<mark>Appeared on: 2023-08-22</mark> -  _Software for JWST/NIRSpec MSA modelling (slit losses, 1D LSFs and 2D model fitting) publicly available at this https URL_

</div>
<div id="authors">

<mark>A. d. Graaff</mark>, et al. -- incl., <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** We explore the kinematic gas properties of six $5.5<z<7.4$ galaxies in the JWST Advanced Deep Extragalactic Survey (JADES), using high-resolution JWST/NIRSpec multi-object spectroscopy of the rest-frame optical emission lines [ O ${\sc iii}$ ] and H $\alpha$ . The objects are small and of low stellar mass ( $\sim 1 $ kpc; $M_*\sim10^{7-9} \Msun$ ), less massive than any galaxy studied kinematically at $z>1$ thus far. The cold gas masses implied by the observed star formation rates are $\sim 10\times$ larger than the stellar masses. We find that their ionised gas is spatially resolved by JWST,  with evidence for broadened lines and spatial velocity gradients. Using a simple thin-disc model, we fit these data with a novel forward modelling software that accounts for the complex geometry, point spread function, and pixellation of the NIRSpec instrument. We find the sample to include both rotation- and dispersion-dominated structures, as we detect velocity gradients of $v(\re)\approx100-150 \kms$ , and find velocity dispersions of $\vdisp\approx 30-70 \kms$ that are comparable to those at cosmic noon. The dynamical masses implied by these models ( $\Mdyn\sim10^{9-10} \Msun$ ) are larger than the stellar masses by up to a factor 40, and larger than the total baryonic mass (gas + stars) by a factor of $\sim 3$ . Qualitatively, this result is robust even if the observed velocity gradients reflect ongoing mergers rather than rotating discs.  Unless the observed emission line kinematics is dominated by outflows, this implies that the centres of these galaxies are dark-matter dominated or that star formation is $3\times$ less efficient, leading to higher inferred gas masses.

</div>

<div id="div_fig1">

<img src="tmp_2308.09742/./figures/sample_cutouts.png" alt="Fig6" width="100%"/>

**Figure 6. -** Sample of six spatially-resolved high-redshift objects in JADES. Left panels show cutouts of the emission lines in the 2D rectified and combined spectra obtained with the high-resolution G395H grating. Negatives in the cutouts are the result of the background subtraction method used. Right panels show NIRCam image cutouts for each object (JADES, FRESCO), for the band that most closely resembles the emission line morphology (Section $\re$f{sec:nircam}). The 3-shutter slits and 3-point nodding pattern used result in an effective area of 5 shutters: the shutter encompassing the source is shown in orange, and the shutters used for background subtraction are shown in purple.   (*fig:sample_overview*)

</div>
<div id="div_fig2">

<img src="tmp_2308.09742/./figures/z_vs_sigma_withsubmm.png" alt="Fig1" width="100%"/>

**Figure 1. -** Velocity dispersion of the ionised gas as a function of redshift. The dashed line shows the fit from [Übler, Genzel and Wisnioski (2019)]() for ionised gas at $0.6<z<2.6$ extrapolated to higher redshifts, while circles show results from a selection of ground-based IFU surveys in the near-infrared  ([Turner, Cirasuolo and Harrison 2017](), [Renzini and Mancini 2018]())  and squares the results from ground-based near-infrared MOS data \citep[the resolved and aligned sample of][]{Price2020}. Blue triangles show results from various studies with ALMA, which measure the kinematics of the cold gas for massive, dusty star-forming galaxies  ([Neeleman, et. al 2020](), [Rizzo, Vegetti and Powell 2020](), [Fraternali, Karim and Magnelli 2021](), [Lelli and Fraternali 2021](), [Rizzo, et. al 2021](), [Herrera-Camus and Price 2022](), [Parlanti, Carniani and Pallottini 2023]()) . The high-redshift objects observed with JWST are dynamically approximately equally turbulent to the observations of more massive galaxies at cosmic noon, and do not follow the trend of increasing $\vdisp$ with redshift observed at $z<4$. (*fig:vdisp*)

</div>
<div id="div_fig3">

<img src="tmp_2308.09742/./figures/exposure_models.png" alt="Fig8" width="100%"/>

**Figure 8. -** Example of the fitting procedure for object JADES-NS-00016745 (Fig. $\re$f{fig:sample_overview}). Although the final combination of all exposures (left) was used for our initial visual inspection and sample selection, the pixels in this spectrum are highly correlated. Instead of using this combined spectrum, we simultaneously fit to all individual exposures obtained. In the case of JADES-NS-00016745 two exposures were taken per nodding position, resulting in six independent measurements for one 3-point nodding pattern with NIRSpec. To combat the undersampled PSF of NIRSpec, we perform our modelling in the detector plane, propagating parametric models to the exact same location on the detector as the observed data. The likelihood is then computed from the combination of all residual images. Pixels flagged by the reduction pipeline as affected by cosmic rays are masked and shown in grey.  (*fig:method*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.09742"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

249  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
